In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# import file
data_df = pd.read_csv("/Users/Ingo/house-prices/house-prices.csv")
print("data_df shape:", data_df.shape)
data_df.head()

In [ ]:
# drop Order and PID as they hold no informational value for the model and can only distort analysis
data_df.drop(["Order", "PID"], axis = 1, inplace = True)

# get overview of entries, data types, NaN
data_df.info(verbose = True, null_counts = True)

It is important to separate test and training data because the different sets serve different purposes:
#### Training Data:
* Is used to fit the model and calculate coefficients which are then used to predict target values based on the later entered test data. 
    * X_tr: These values are the features in our training set that are used to explain the target variable. In this data set, we can perform transformation, feature engineering, encode ordinal values, clean outliers, drop duplicates, etc. etc.
    * y_tr: This is the target variable which we usually want to predict. We can also drop outliers here if we drop them in the X_tr set. This is done in order to make keep the correct indices. If one row is dropped in X_tr, this value should also be dropped here. If outliers are kept here, we encompass this outlier in the training of the model. We wan to avoid this to make the model more robust and enable it to generalize better. 
    
#### Test Data:
* Uses the coefficients from the training data to calculate predictions for the target variable.
    * X_te: These values are put into the model to generate y-predictions.
        * We can transform/encode these values, but we should not drop outliers or duplicates from here as this is real data that our model should handle in the way it is entered (aside from encoding ordinal values etc.)
    * y_te: These are the "real" values used to test the predictions against.
        * y_te should not be touched in term of outliers or duplicates because these values act as a reference to test our predictions against. If we e.g. log-transformed y_tr, we can also log-transform y_te.
        * Note: We need to re-transform the predictions, otherwise we have log-transformed predictions
        

Because the `SalePrice` variable is very spread out (as we will later see), I think a good starting point for a train/test split is to assign a 70% portion to the train set. The coefficients are build on this data, so it is important to enable the coefficents to learn on as much data as possible. At the same time is it desirable to not feed to much data and have too few entries in the test set, so that we can't cover enough test situations and maybe won't know about flaws when testing our predictions. Therefore, I think a 50%-50% spread provides too little learning opportunity after cleaning for the coefficients, so I went with a 70%-30% spread, which leaves us with 1432 entries for the training set, and 729 entries for the test set - so about 2:1.

In [ ]:
# get train_df, test_df to separate cleaning and encoding
from sklearn.model_selection import train_test_split

train_df_org, test_df_org = train_test_split(data_df, train_size = 0.7, test_size = 0.3, random_state = 0)

# work on a copy of those two dfs, otherwise warning is raised
train_df = train_df_org.copy()
test_df = test_df_org.copy()

Now I'm going to clean `train_df`, as well as transform/encode `train_df` and `test_df`

In [ ]:
# check the columns that have NaN in train_df values to fill them 
train_df.isnull().sum()[40:]

In [ ]:
# columns that have NaN in them
train_df_nan_cols = ["Lot Frontage", "Alley", "Mas Vnr Type", "Mas Vnr Area", "Bsmt Qual", "Bsmt Cond",
                       "Bsmt Exposure", "BsmtFin Type 1", "BsmtFin SF 1", "BsmtFin Type 2", "BsmtFin SF 2",
                       "Bsmt Unf SF", "Total Bsmt SF", "Electrical", "Bsmt Full Bath", "Bsmt Half Bath",
                      "Fireplace Qu", "Garage Type", "Garage Yr Blt", "Garage Finish", "Garage Cars",
                       "Garage Area", "Garage Qual", "Garage Cond", "Pool QC", "Fence", "Misc Feature"]

In [ ]:
train_df.loc[:, train_df_nan_cols][:4] # this cell is just used to check different column entries in the df

#### Continuous Features:
Will be filled with the median of the column. Median avoids being skewd into either direction by not being affected by extremes, so it is preferable to the mean.
* `Lot Frontage`
* `Mas Vnr Area`, "Mas Vnr Type" also has the same 15 entries missing, and as I fill those with `"None"` (the string "None"), I'll fill these missing values with `0` so that it makes sense
* `BsmtFin SF 1`
* `BsmtFin SF 2`
* `Bsmt Unf SF`
* `Total Bsmt SF` 
* `Garage Area`
* `Garage Yr Blt` is a discrete feature, will be filled with median


#### Nominal/Ordinal Features:
Often `NaN` means that the feature hasn't been build for that house, i.e. NaN in `Alley` means that there is no alley access to that property. I will fill those columns with `missing`, which enables following analysis to recognize the fact that there is no such feature at these houses.
* `Alley`
* `Mas Vnr Type`, only 15 missing, so not worth it to create a new category. Also, `None` already exists as a string, so instead of "missing", I'll fill with `None`
* `Bsmt Qual`
* `Bsmt Cond`
* `Bsmt Exposure`
* `BsmtFin Type 1`
* `BsmtFin Type 2`
* `Electrical` is missing only 1 value, so I'll fill the most common value in
* `Bsmt Full Bath` is missing only 2 values, so I'll fill in the most common value
* `Bsmt Half Bath` is missing only 2 values, so I'll fill in the most common value
* `Fireplace Qu`
* `Garage Type` 
* `Garage Finish`
* `Garage Cars` only missing 1, fill in most common value
* `Garage Qual`
* `Garage Cond`
* `Pool QC`
* `Fence`
* `Misc Feature`

In [ ]:
# fill continuous features
train_df.loc[:, "Mas Vnr Area"] = train_df["Mas Vnr Area"].fillna(0) # Mas Vnr Area filled with separate value
# fill in same value for test set
test_df.loc[:, "Mas Vnr Area"] = test_df["Mas Vnr Area"].fillna(0)

# group features that will be transformed
cont_feature_list = ["Lot Frontage", "BsmtFin SF 1", "BsmtFin SF 2", "Bsmt Unf SF", "Total Bsmt SF",
                     "Garage Area", "Garage Yr Blt"]
mode_feature_list = ["Mas Vnr Type", "Electrical", "Bsmt Full Bath", "Bsmt Half Bath", "Garage Cars"]
missing_col_list = ["Alley", "Bsmt Qual", "Bsmt Cond", "Bsmt Exposure", "BsmtFin Type 1", "BsmtFin Type 2",
                    "Fireplace Qu", "Garage Type", "Garage Finish", "Garage Qual",
                    "Garage Cond", "Pool QC", "Fence", "Misc Feature"] # fill in string "missing"

from sklearn.impute import SimpleImputer

# don't use ColumnTransformer because I still want DataFrames as a result to continue working
imputer1 = SimpleImputer(strategy = "median") # for continuous features
imputer2 = SimpleImputer(strategy = "most_frequent") # for the mode features
imputer3 = SimpleImputer(strategy = "constant", fill_value = "missing") # for missing_col_list

# fit on training data, then transform training data, then fill in training data into test set
train_df_cont_imputed = imputer1.fit_transform(train_df[cont_feature_list])
test_df_cont_imputed = imputer1.transform(test_df[cont_feature_list])

train_df_mode_imputed = imputer2.fit_transform(train_df[mode_feature_list])
test_df_mode_imputed = imputer2.transform(test_df[mode_feature_list])

train_df_missing_imputed = imputer3.fit_transform(train_df[missing_col_list])
test_df_missing_imputed = imputer3.fit_transform(test_df[missing_col_list])

In [ ]:
# assign values back to train and test set
train_df.loc[:, cont_feature_list] = train_df_cont_imputed
train_df.loc[:, mode_feature_list] = train_df_mode_imputed
train_df.loc[:, missing_col_list] = train_df_missing_imputed

test_df.loc[:, cont_feature_list] = test_df_cont_imputed
test_df.loc[:, mode_feature_list] = test_df_mode_imputed
test_df.loc[:, missing_col_list] = test_df_missing_imputed

In [ ]:
print("train_df number of missing values:", train_df.isnull().sum().sum())
print("test_df number of missing values:", test_df.isnull().sum().sum())

In [ ]:
# get an overview of numerical columns
train_df.describe().T

The descriptive overview provides a broad sense of minimum values, maximum values, and in what range the values are.

We can see, especially for the target variable `SalePrice` that median > median with a difference of over \$20,000, which means that the variable has some extremely high selling prices, biasing prediction accuracy in favor of expensive sales. We will have to transform the variable later on, e.g. with a log-transformation, to make it more Gaussian-like.

In [ ]:
train_df.info()

## Ordinal Features
I will deal with ordinal features in two ways:
* If all values are available, i.e. no `missing` used to fill NaNs, I will encode the features numerically, with the best feature having the highest number.
* If I filled NaNs with `missing`, it would distort the logic behind "missed to enter" and "not build" when there is an ordinal number ranking, e.g. in cases when I assign a 0 to "missing". For instance, having missed to enter the `Sale Condition` is different then no pool being available on the property.

In [ ]:
# find ordinal columns and distinguish between "missing" (m) and not missing (nm)
# some ordinal columns are already numerically encoded, they are left out
nm_ordinal_cols = ["Lot Shape", "Utilities", "Land Slope", "Exter Qual", "Exter Cond", "Heating QC",
                  "Electrical", "Kitchen Qual", "Functional",
                   "Paved Drive"] # ordinal - according to documentation.txt

m_ordinal_cols = ["Bsmt Qual", "Bsmt Cond", "Bsmt Exposure", "BsmtFin Type 1", "BsmtFin Type 2",
                 "Fireplace Qu", "Garage Finish", "Garage Qual", "Garage Cond", "Pool QC",
                  "Fence"] # ordinal - according to documentation.txt

In [ ]:
# create dictionaries to map nm_ordinal_cols' values to numeric values
lot_shape_dict = {"Reg" : 4, "IR1" : 3, "IR2" : 2, "IR3" : 1}
utilities_dict = {"AllPub" : 4, "NoSewr" : 3, "NoSeWa" : 2, "ELO" : 1}
land_slope_dict = {"Gtl" : 3, "Mod" : 2, "Sev" : 1}
exter_qual_dict = {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}
exter_cond_dict = {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}
heating_qc_dict = {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}
electrical_dict = {"SBrkr" : 5, "FuseA" : 4, "FuseF" : 3, "FuseP" : 2, "Mix" : 1}
kitchen_qual_dict = {"Ex" : 5, "Gd" : 4, "TA" : 3, "Fa" : 2, "Po" : 1}
functional_dict = {"Typ" : 8, "Min1" : 7, "Min2" : 6, "Mod" : 5, "Maj1" : 4, "Maj2" : 3, "Sev" : 2,
                   "Sal" : 1}
paved_drive_dict = {"Y" : 3, "P" : 2, "N" : 1}

In [ ]:
# map the values onto train_df columns
dict_list = [lot_shape_dict, utilities_dict, land_slope_dict, exter_qual_dict, exter_cond_dict,
            heating_qc_dict, electrical_dict, kitchen_qual_dict, functional_dict, paved_drive_dict]

for col, dictionary in zip(nm_ordinal_cols, dict_list):
    train_df.loc[:, col] = train_df[col].map(dictionary)


# map values in test_df, map() still works here as there is no other value except NaN in test_df
for col, dictionary in zip(nm_ordinal_cols, dict_list):
    test_df.loc[:, col] = test_df[col].map(dictionary)

In [ ]:
# check whether results worked
train_df[nm_ordinal_cols].head() # worked
test_df[nm_ordinal_cols].head() # worked

There is a nominal variable which is encoded as numbers but isn't interpretable as numbers, i.e. there is no amount described or a ranking. I'll change it to letters to still keep it appropriate for one hot encoding later on.

The column is:
* `MS SubClass`

In [ ]:
# get unique values of column (there are/were no NaNs in this column)
train_df["MS SubClass"].unique()

In [ ]:
# create dictionary to map values
ms_subclass_dict = {
    20 : "A",
    30 : "B",
    40 : "C",
    45 : "D",
    50 : "E",
    60 : "F",
    70 : "G",
    75 : "H",
    80 : "I",
    85 : "J",
    90 : "K",
    120 : "L",
    150 : "M",
    160 : "N",
    180 : "O",
    190 : "P"
}

# map values
train_df.loc[:, "MS SubClass"] = train_df["MS SubClass"].map(ms_subclass_dict)
test_df.loc[:, "MS SubClass"] = test_df["MS SubClass"].map(ms_subclass_dict)

Let's visualize the distribution of numeric columns with jointplots (combination of histograms and scatter plots) in order to see outliers and distributions. To do this, I'll visualize all features marked _continuous_ in the documentation.txt file.  

In [ ]:
import seaborn as sns
continuous_features = ["Lot Frontage", "Lot Area", "Mas Vnr Area", "BsmtFin SF 1", "BsmtFin SF 2",
                      "Bsmt Unf SF", "Total Bsmt SF", "1st Flr SF", "2nd Flr SF", "Low Qual Fin SF",
                      "Gr Liv Area", "Garage Area", "Wood Deck SF", "Open Porch SF", "Enclosed Porch",
                      "3Ssn Porch", "Screen Porch", "Pool Area", "Misc Val", "SalePrice"]

for col in continuous_features:
    sns.jointplot(train_df["SalePrice"], col, data = train_df, height = 6, s = 1)
    plt.show()

At this point, we can already see that there are many extreme values, so cleaning is needed to fit a model that is supposed to lead to robust predictions.

Furthermore, there are a lot of zeros, making it a subpopulation that needs to be considered on its own, i.e. removed before calculating standard deviations

It doesn't seem like there are any negative values, but I'll check for them anyway.

I'll check for negative values in both train- and test-set. These would be wrongly entered numbers by mistake as no value of these columns could logically be negative. If there are negative numbers, I'd even clean those in the test_df as these are not realistic values to consider for entry. 

In [ ]:
print("Number of negative values in train_df:", (train_df[continuous_features] < 0).sum().sum())
print("Number of negative values in test_df:", (test_df[continuous_features] < 0).sum().sum())

In [ ]:
# group together features with a majority of zeros
maj_zero_features = ["Mas Vnr Area", "BsmtFin SF 1", "BsmtFin SF 2", "2nd Flr SF", "Low Qual Fin SF",
                    "Wood Deck SF", "Open Porch SF", "Enclosed Porch", "3Ssn Porch", "Screen Porch",
                    "Pool Area", "Misc Val"]

In [ ]:
# plot histograms of continuous data excluding zeros to compare later results to this
fig, axes = plt.subplots(nrows = 3, ncols = 4, figsize = (15, 8))

for col, ax in zip(maj_zero_features, axes.ravel()):
    ax.hist(train_df[col][train_df[col] != 0], bins = 20)
    ax.set_title(col)

fig.suptitle("Distribution of Majority Zero Continuous Features, Excluding Zeros, Uncleaned",
             y = 1.05, size = 20)
fig.tight_layout()
plt.show()

From these distributions we can see that some variables are close to Gaussian-like, but they are pushed to the left (e.g. `Lot Frontage`, `1st Flr SF`) due to extreme positive outliers. These need to be cleaned.

In addition to that, there are multiple features that have overwhelming numbers of 0 (e.g. `2nd Flr SF`, `Wood Deck SF`), so they need to be analyzed on its own while excluding the 0 subpopulation.

Then there are columns that need to be cleaned from outliers, but this step needs to happen after the Zero-subpopulation is excluded. That would be the case e.g. `Open Porch SF` or `Enclosed Porch`. 

Also, the `SalePrice` variable seems to have some extreme values which will cause biased predictions in favor of high-cost houses as mean > median. 

--> Training data needs to be cleaned
____

First, I'll calculate the ratio of $\frac{\text{Zeros}}{\text{all entries}}$, if ratio is at least 10%, I'll exclude the zeros. Otherwise I treat the zeros as regular, common values

In [ ]:
cont_train_df = train_df[continuous_features].copy() # create copy only with continuous variables
((cont_train_df == 0).sum(axis = 0) / (cont_train_df.count(axis = 0))) * 100

In [ ]:
maj_zero_df = train_df[maj_zero_features].copy() # create df only with the features that need adjustment

# log-transform the maj_zero_df to make z-scores meaningful
maj_zero_df_log = maj_zero_df.applymap(np.log1p)

# replace 0 with np.nan, inplace = True
maj_zero_df_log.replace(to_replace = 0, value = np.nan, inplace = True)

# create function to calculate z-scores, ignores NaNs during calculation and maintains NaN values in cols
def z_score(df):
    mean = df.mean()
    std = df.std()
    return ((df - mean) / std)

# calculate z-scores for majoriy zero columns, but without letting Zeros affect the z-score as 0 are NaN
z_maj_zero_df_log = maj_zero_df_log.apply(z_score)
print("Mas Vnr Area mean should be ~0, test:", z_maj_zero_df_log["Mas Vnr Area"].mean())
print("Mas Vnr Area std should be ~1, test:", z_maj_zero_df_log["Mas Vnr Area"].std())

In [ ]:
# plot the z-scaled values to see distribution of these columns
fig, axes = plt.subplots(nrows = 3, ncols = 4, figsize = (15, 8))

for col, ax in zip(maj_zero_features, axes.ravel()): 
    ax.hist(z_maj_zero_df_log[col][~np.isnan(z_maj_zero_df_log[col])], bins = 20) # plot all values except nan
    ax.set_title(col)                                                     # avoids RuntimeWarning  

fig.suptitle("Z-Score Distribution of Majority Zero Continuous Columns, Zeros Excluded",
             y = 1.05, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

We can see that there are several features which have z-scores over 3 std. Those values should be cleaned in the training set to give the model more robust coefficients.

In [ ]:
# check how many values per column are > 3 std
print("train_df shape before drops:", train_df.shape)
print("number of features per column to drop:\n", (np.abs(z_maj_zero_df_log) > 3).sum(axis = 0))
print("number of features to drop in total:", (np.abs(z_maj_zero_df_log) > 3).sum(axis = 0).sum())

In [ ]:
# drop values > 3 std from train_df inplace = True, ignore errors when labels don't exist,
    # i.e. value has already been dropped
for col in maj_zero_features:
    # get indices to drop for each column in a variable for each loop
    index_to_drop = z_maj_zero_df_log.loc[np.abs(z_maj_zero_df_log[col]) >= 3].index
    # drop the indices each loop
    train_df.drop(index = index_to_drop, inplace = True, errors = "ignore")

print("train_df shape after drops:", train_df.shape)

In [ ]:
# visualize results again
fig, axes = plt.subplots(nrows = 3, ncols = 4, figsize = (15, 8))

for col, ax in zip(maj_zero_features, axes.ravel()):
    ax.hist(train_df[col][train_df[col] != 0], bins = 20) # plot without zeros
    ax.set_title(col)

fig.suptitle("Distribution of Majority Zero Continuous Features, Cleaned, Zeros Excluded",
             y = 1.05, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

We can see that the distribution is much closer together now that every value with a z-scaled std > 3 has been dropped.

Now I'll clean the columns that didn't have a majority of zeros in them.

In [ ]:
# calculate ratio of zeros in continuous columns again to see which columns to clean now
((cont_train_df == 0).sum(axis = 0) / cont_train_df.count(axis = 0)) * 100

In [ ]:
# build list of features
to_clean = ["Lot Frontage", "Lot Area", "Bsmt Unf SF", "Total Bsmt SF", "1st Flr SF", "Gr Liv Area",
           "Garage Area"]

# create separate df to apply z_score function to
z_to_clean_df = train_df[to_clean].copy()

# log-transform numbers to make them meaningful for z-scores
z_to_clean_df_log = z_to_clean_df.applymap(np.log1p)

# apply z-scaling
z_to_clean_df_log = z_to_clean_df_log.apply(z_score) # here we calculate z_scores INCLUDING the zeros

# number of entries that will be dropped per column
print("train_df shape before drops:", train_df.shape)
print("number of entries to drop\n", (np.abs(z_to_clean_df_log) > 3).sum())
print("number of entries in total to drop:", (np.abs(z_to_clean_df_log) > 3).sum().sum())

Despite log-transformation there are a lot of skewed values in this data, so we have to drop 255 values. It is interesting to see the z-scaled distribution of these features.

In [ ]:
fig, axes = plt.subplots(nrows = 2, ncols = 4, figsize = (15, 8))

for col, ax in zip(z_to_clean_df_log.columns, axes.ravel()):
    ax.hist(z_to_clean_df_log[col], bins = 20)
    ax.set_title(col)
    
plt.suptitle("Distribution of Z-Scaled Non-Maj. Zero Continuous Features", y = 1.03, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# drop values > 3 std in training set, ignore errors from previously dropped indices, drop inplace = True
for col in to_clean:
    # group indices of a column in a variable for each loop
    index_to_drop = z_to_clean_df_log.loc[np.abs(z_to_clean_df_log[col]) > 3].index
    # drop the indices from train df
    train_df.drop(index = index_to_drop, inplace = True, errors = "ignore")
    
print("train_df shape after drops:", train_df.shape)

# visualize the results
fig, axes = plt.subplots(nrows = 2, ncols = 4, figsize = (15, 5))

for col, ax in zip(to_clean, axes.ravel()):
    ax.hist(train_df[col], bins = 20)
    ax.set_title(col)

fig.suptitle("Distribution of Non-Majority Zero Continuous Features, Cleaned", y = 1.05, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# final descriptive summary of data
train_df[continuous_features].describe().T

We can see that the features are now much less spread out, and they will therefore produce more robust predictions.

I won't clean SalePrice directly. The cleaning of the feature columns has already cleaned out some of the extreme values. We can observe it because the maximum value is only \\$615,000 now in comparison to over \\$700,000 before. However, it is noteworthy that both the mean and median increased here after cleaning. This is possible because the cleaning process dropped some extremely low values, too, so the overall distribution increased to a higher level.

In a later step, I apply a log-transformation of the feature and target variables to make them more Gaussian-like and by doing that, curve the extremely expensive house sales.

We can see, especially here in the non-majority zero features, that some features have a Gaussian-like distribution. To get a better overview of the state of cleaning, I'll visualize all features again with jointplots.

In [ ]:
# jointplots of all continuous features

for col in continuous_features:
    sns.jointplot(train_df["SalePrice"], train_df[col], height = 6, s = 1)
    plt.show()

Still, a lot of continuous features are far from being nearly normally distributed. To get closer to a preferred normal distribution during the entire modeling process, I will use the log-transformation as it centers the value more closely to what is preferred when building models.

For the log-transformation, I'll use the numpy `log1p` function, which always adds 1 to the value, so that all columns with a big zero subpopulation will again result to 0 because $0 + 1 = 1$, $log{(1)} = 0$.

In [ ]:
# log-transformation of continuous features
log_train_df = train_df[continuous_features].copy() # create copy for log value df
log_train_df = log_train_df.applymap(np.log1p)

# plot Gaussian curve on top of transformed features
from scipy.stats import norm

# visualize log transformed feature distribution
fig, axes = plt.subplots(nrows = 4, ncols = 5, figsize = (20, 12))

for col, ax, position in zip(continuous_features, axes.ravel(), np.arange(20)):
    # plot a distplot, i.e. histogram, no kernel density estimate, but fit a normal distribution on top
    # to see how much our transformation differs from the normal distribution
    ax = sns.distplot(log_train_df[col], bins = 20, hist = True, kde = False, fit = norm, ax = 
                     axes[position // 5][position % 5]) # encodes the position on the grid
    ax.set_title(col)

fig.suptitle("Log-Transformed Continuous Features", y = 1.05, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

We can see that the transformed features are now more evenly placed in the center next to the zero subpopulation.

Especially the `SalePrice` variable has been centered nicely, which is important when trying to get to stable predictions. 

___
Now that the cleaning has been done, I'll group all variables that will need OneHotEncoding in the intermediate and complex model, maybe even in the simple model, depending on the variable choice later on.

I'll apply OHE to select features that are discrete but not ordinal (or have the "missing" string in them). During the cleaning process I encoded all ordinal features without "missing" values in a ranked order, so there is an internal logic that is regression-readable to the data. Ordinal features with the "missing" string will be OHE because missing features is different than the condition _poor_, and should therefore be read differently in a regression calculation.

In general, all other features except dates will be one-hot-encoded to make them readable in a regression. Dates, e.g. year built or year renovated, are fine as they are because they are sorted, implying ordinal properties. Other features similar to year, like `TotRmsAbvGrd` or `Bsmt Full Bath` are discrete but will also not be OHE as the different numbers imply a ranking.

`Mo Sold` will be OHE because there can be trends during certain months that are relevant for predictions, but a house sold in month 12 shouldn't be numerically encoded as being above a house sold in month 06. `Yr Sold` on the other hand will stay non-OHE because having sold in a later year can imply being more expensive. Macroeconomic trends can be observed here, so a number of the year can have ramifications to the target variable.

In [ ]:
# group cols which will be OHE
OHE_cols = ["MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood",
           "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl",
           "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", "Foundation", "Bsmt Qual", "Bsmt Cond",
           "Bsmt Exposure", "BsmtFin Type 1", "BsmtFin Type 2", "Heating", "Central Air", "Fireplace Qu",
           "Garage Type", "Garage Finish", "Garage Qual", "Garage Cond", "Pool QC", "Fence",
            "Misc Feature", "Mo Sold", "Sale Type", "Sale Condition"]

This marks the end of my cleaning steps. Because there will be 3 different models, I won't transform all columns at this stage because analyzing which features to use in the models is more difficult when there are already multiple OHE variables in the DataFrames.

# Model Evaluation
Each model will be evaulated by the rescaled-to-dollars MAE and the RSS, which will be used to calculate $R^2$, which is only used to calculate the adjusted R2 $\bar{R}^2$. I will explain my reasoning for adj. R2 below. 

Calculate baseline model: For the baseline model I will use the median of the `SalePrice`. The median is more robust to outliers - even when the target variable is log- or sqrt-transformed - and serves, therefore, as a more reliable baseline when all extreme sale prices are included.

In parallel, I will still calculate a mean baseline in order to calculate the adjusted R2.

## R2 vs adj. R2

Adjusted R2 formula:
$$
\bar{R}^2 = 1-(1-R^2)\frac{n-1}{n-p-1}, \text{with n = number of observations, p = number of features}
$$

### Problems with R2:
R2 is increasing monotonically as the number of explanatory variables is increasing. This means that we automatically get a higher R2 when more features are taken into account, e.g. through OHE or expanding the model from simple to intermediate to complex. Of course, more variables can and will include meaningful variables that do, indeed, increase the model explanatory power, but there could also be variables which increase the R2 with no causal relation or even completely by chance. To counteract this problem, adjusted R2 penalizes the R2 score as extra variables are added to the model. Since every model in this course will have more than one feature, I think it is appropriate to use adj. R2.

### Problem with Adjusted R2 and Regularized Coefficients Models
Adjusted R2 gets distorted and is therefore limited in its interpretation when coefficients are already penalized in the model. This is especially relevant for my intermediate and complex model, in both of which I will use regularization due to overfitting. 

Nonetheless, I will calculate the adj. R2 metric for all three model, and will point out that the MAE metrics may be more representative of model performance. 
___
# Simple Model with 2 Features
Now I will search for the two features to incorporate into the simple model. These two features will be selected by calculating the correlation between each feature and the target variable.

Since I encoded ordinal features into ranked numbers, I can include them in the correlation calculation aswell.

In [ ]:
# group variables that we want to test for correlation, start with all continuous features
corr_features = ['Lot Frontage', 'Lot Area', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF',
                 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area',
                 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch',
                 'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice', "Overall Qual", "Overall Cond",
                 "Year Built", "Year Remod/Add", "Exter Qual", "Exter Cond", "Heating QC", "Electrical",
                 "Bsmt Full Bath", "Bsmt Half Bath", "Full Bath", "Half Bath", "Bedroom AbvGr",
                 "Kitchen AbvGr", "Kitchen Qual", "TotRms AbvGrd", "Functional", "Fireplaces",
                 "Garage Yr Blt", "Garage Cars", "Yr Sold"]


# separate features from target variable
corr_X_tr = train_df[corr_features].drop("SalePrice", axis = 1)

# target variable, apply log-transformation
corr_y_tr = np.log(train_df.SalePrice)


# apply log-transformation to continuous features of X_tr
continuous_features.remove("SalePrice") # remove SalePrice for this slicing
corr_X_tr.loc[:, continuous_features] = corr_X_tr[continuous_features].applymap(
    np.log1p) # log-transform

In [ ]:
# create a df with both log-transformed features and log-transformed target variable
corr_df = pd.concat((corr_X_tr, corr_y_tr), axis = 1) # concatenate log-transformed X_tr, log-transformed y_tr
corr_df = corr_df.corr() # calculate the correlation matrix


corr_df.loc[corr_df["SalePrice"] > 0.5, "SalePrice"].sort_values(ascending = False)[:3]

The results here confirm the popular intuition that house prices are (among others) largely determined by the general quality of the house and the above ground living space. Because of this, I'll build the first model with these 2 variables.

Note: The neighborhoods feature (which is generally known to drive prices) isn't in a format in which it would make sense to calculate correlations. Even if it were OHE, it still produces different results than correlations among continuous features.

In [ ]:
# visualize distribution of both features against the target variable
ax1 = sns.violinplot(x = corr_X_tr["Overall Qual"], y = corr_y_tr,
                     ) # discrete feature, needs violinplots

ax2 = sns.jointplot(x = corr_X_tr["Gr Liv Area"], y = corr_y_tr, s = 5,
                   ) # continuous feature can be scatter plotted
ax1.set_title("Distribution of Overall House Quality and Sale Prices")
ax2.fig.suptitle("Distribution of Above Ground Living Area and Sale Prices", y = 1.03)
ax2.fig.tight_layout() # avoids overlapping labels
plt.show()

For the violin plots we can see a linear trend that with rising overall quality of the house, the sale price is also rising. There are, however, outliers in the quality ranks, meaning that some houses are ranked as being low quality but have a relatively high sale price for that quality.

There are also outliers between living area and sale prices, but in general there is a strong overlap for the majority of data. 

For this reason, I choose to fit a linear regression model without any regularization.

In [ ]:
# create linear regression model with Overall Qual as feature 1, Gr Liv Area as feature 2
# m1 --> model1
m1_X_tr = corr_X_tr.loc[:, ["Overall Qual", "Gr Liv Area"]].values # create feature array
m1_X_te_raw = test_df.loc[:, ["Overall Qual", "Gr Liv Area"]].values # create untransformed X_te

# create another X_te with transformations
m1_X_te = np.c_[m1_X_te_raw[:,0], np.log1p(m1_X_te_raw[:,1])]

m1_y_tr = corr_y_tr.values # this is already log-transformed
m1_y_te = np.log(test_df.SalePrice).values

# check for collinear features
# add array of ones to be able to recognize linear combinations
m1_X1_tr = np.c_[np.ones(shape = m1_X_tr.shape[0]), m1_X_tr]

# check rank of matrix
m1_rank = np.linalg.matrix_rank(m1_X1_tr)
print("matrix rank of model_1 features:", m1_rank) # it has full rank, i.e. no collinear features

# check for nearly collinear features with condition number
m1_condition = np.linalg.cond(m1_X1_tr)
print("condition number of model_1:", m1_condition) # number is low, i.e. no nearly collinear features

In [ ]:
from sklearn.linear_model import LinearRegression
m1_lr = LinearRegression() # create object
m1_lr.fit(m1_X_tr, m1_y_tr) # fit training data, this calculates the coefficients

from sklearn.metrics import mean_absolute_error as MAE

# make predictions based on X_te set
m1_pred = m1_lr.predict(m1_X_te)
# calculate MAE from the re-transformed X_te predictions against re-transformed y_te to have results in $
print("simple model MAE is ${:,.0f}".format(MAE(np.exp(m1_y_te), np.exp(m1_pred))))

Considering that we only used 2 out of 81 features, the error score seems reasonably good. On a scale where the median sale price is \\$169,000, an median error of \\$23,479, or around $\frac{23,479}{169,000} = 13,89\%$, is a good result. 

To provide some perspective of why I think this fraction metric is a good comparison (it is based the $R^2$ thinking): If we wanted to predict the sale price of cars, and the median car sale price was \\$25,000, and our MAE was \\$23,000, this wouldn't be a good model as the average prediction almost misses the entire median car sales price. Predictions would therefore be extremely untrustworthy. 

Nevertheless, we still can improve this model by taking more features, transformations, encodings into the model, i.e. creating a more sophisticated model.

To calculate the $\text{adj.} \hspace{0.05cm} R^2$ scores later on, I'll calculate the model RSS now.

In [ ]:
# def function to calculate RSS
def RSS(y, y_pred):
    return np.sum(np.square(y - y_pred))
m1_rss = RSS(m1_y_te, m1_pred)
print("m1 rss is {:.2f}".format(m1_rss))

As long as the baseline RSS stays in this format, we don't have to re-transform the RSS values because the $R^2$ value is a ratio.
___

## Preprocess the separate test set
This process mirrors the preprocessing steps of the test set (validation set)

In [ ]:
test_predictions_df = pd.read_csv("/Users/Ingo/house-prices/house-prices-test.csv")

# fill in missing values with values from train set
test_predictions_df.loc[:, "Mas Vnr Area"] = test_predictions_df["Mas Vnr Area"].fillna(0)

test_predictions_median_imputed = imputer1.transform(test_predictions_df[cont_feature_list])
test_predictions_mode_imputed = imputer2.transform(test_predictions_df[mode_feature_list])
test_predictions_missing_imputed = imputer3.transform(test_predictions_df[missing_col_list])

# reassign values to df from numpy arrays
test_predictions_df.loc[:, cont_feature_list] = test_predictions_median_imputed
test_predictions_df.loc[:, mode_feature_list] = test_predictions_mode_imputed
test_predictions_df.loc[:, missing_col_list] = test_predictions_missing_imputed

# encode necessary features of test data
# map values in test df, map() still works here as there is no other value except NaN in test_df
for col, dictionary in zip(nm_ordinal_cols, dict_list):
    test_predictions_df.loc[:, col] = test_predictions_df[col].map(dictionary)
    
# encode MS SubClass
test_predictions_df.loc[:, "MS SubClass"] = test_predictions_df["MS SubClass"].map(ms_subclass_dict)

In [ ]:
m1_test_df = test_predictions_df[["PID", "Overall Qual", "Gr Liv Area"]].copy() # slice copy to work on

# log-transofrm the continuous feature, concatenate both features
m1_test_features = np.c_[m1_test_df["Overall Qual"], np.log1p(m1_test_df["Gr Liv Area"])]

# make predictions with lr object fitted with training data
m1_test_pred = m1_lr.predict(m1_test_features)

# concatenate PID
m1_test_file = pd.DataFrame({
    "PID" : m1_test_df["PID"],
    "SalePrice" : np.round(np.exp(m1_test_pred), 0).astype(int) # reverse log-transformation on m1_y_tr
})

# save the file
m1_test_file.to_csv("/Users/Ingo/house-prices/predictions-simple-model.csv", index = False)

# display examples of predictions
m1_test_file.head()

___
# Intermediate model
To select the 10-20 features to be incorporated into the intermediate model, I will use the forests of trees methodology found in the [scikit-learn feature selection overview.](https://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection) This selection method evaluates the importance of each feature, meaning that we can rank which features are most important in a model. The results will be visualized, so that we can see the relative importance of each feature against all other features.

As the `RandomForestRegressor` cannot work with strings on its own, we have to preprocess and encode/transform the feature DataFrame. Because of this, `RandomForestRegressor` might rank a single OHE column as an important feature in my top 20 list. I will take note of that and in turn put in the affiliated, untransformed column as one of the 10-20 features, and then OHE all of those features again.

Example: Feature 123 is ranked in my top 20 as an important feature. But feature 123 is one of the OHE column of the original column `Neighborhood`, i.e. a 0 or 1 column of one specific neighborhood. In that case, I will put `Neighborhood` as a whole in my model, and when preprocessing that model, I will OHE `Neighborhood` as a whole again. 

In [ ]:
# check whether the two lists for transformations intersect - they shouldn't
set(continuous_features) & set(OHE_cols)

In [ ]:
# create feature and target arrays
X_tr_general = train_df.drop("SalePrice", axis = 1) # not yet transform to arrays
y_tr_general = train_df.SalePrice.values

# encode and transform arrays to make them readable for ExtraTreesClassifier
# import column transformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer

# ignore FutureWarning
import warnings
warnings.simplefilter("ignore", FutureWarning)

# create transformer variables and objects
# drop first to avoid linear dependences and numerically unstable results
ohe_transformer = OneHotEncoder(drop = "first", sparse = False)
log_transformer = FunctionTransformer(np.log1p) # apply log transformation to continuous features

# create ColumnTransformer and input all functions
forest_preprocessor = ColumnTransformer([
    ("OHE", ohe_transformer, OHE_cols), # input OneHotEncoding for OHE cols
    ("log", log_transformer, continuous_features) # input log transformation
], remainder = "passthrough") # let all columns as they are if they are not mentioned in the transformation

# fit and transform features
forest_X_tr = forest_preprocessor.fit_transform(X_tr_general)

# log-transform target variable
y_tr_general = np.log(y_tr_general)

# print shapes
print("shape of X_tr after preprocessing:", forest_X_tr.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# create object, make features * 4 estimators (~250 * 4)
forest = RandomForestRegressor(n_estimators = 1000, random_state = 0)

forest.fit(forest_X_tr, y_tr_general) # fit the object with training data
importances = forest.feature_importances_ # returns the feature importances, the higher the better
std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis = 0) # calculates deviation of each feature
# argsort sorts by default ascending = True, but we want the highest value first, so slice [::-1]
indices = np.argsort(importances)[::-1]

# print results
print("Feature ranking:")
for f in range(20): # print top 20 results
    print("{}. feature {} {:.4f}".format(f + 1, indices[f], importances[indices[f]]))

In [ ]:
# visualize results
plt.figure(figsize = (10, 5))
plt.bar(np.arange(20), importances[indices[:20]], yerr = std[indices[:20]], align = "center")
plt.xticks(np.arange(20), labels = indices[:20])
plt.xlabel("Feature Number")
plt.ylabel("Feature Importance")
plt.title("The 20 Most Important Features According to Random Forest Selection")
plt.show()

We can see that some features have a very high standard deviation in their importances, so it is not _always_ the case that the same feature has the same importance for every prediction.

Currently, we don't know which feature is which, in genereal, however, we can see that feature numbers are high, meaning that they occoured toward the end of the DataFrame and are, therefore, probably log-transformed continuous columns or untransformed ordinal features. The `ColumnTransformer` first applied OHE, so the early columns are OHE features. After that, the `ColumnTransformer` applied and concatenated the log-transformations which are hence at the end of the DataFrame with corresponding high feature numbers.

In [ ]:
# try to get feature names back to see which features are the most impactful
# first, get named_transformers
forest_preprocessor.named_transformers_

In [ ]:
OHE_feature_names = forest_preprocessor.named_transformers_["OHE"].get_feature_names()
len(OHE_feature_names)

We can see that 207 (i.e. index number 0 to 206) of the 251 features are OHE columns. By knowing that, it follows that only 1 feature in the top 20 are OHE features (because only 1 feature is $\leq$ index 206):

In [ ]:
# get the feature name of the OHE column
print("feature name of OHE feature:", OHE_feature_names[153])

* `x24_missing` belongs to `Fireplace Qu` 

We can verify all of this by entering the index `24` into `OHE_cols`.

In [ ]:
# verify the above
print(OHE_cols[24])

As `FunctionTransfomer` has no attribute `get_feature_names`, we have to improvise to get the feature names back. We know that the first 206 indices are OHE columns. Therefore, starting at index 207 until index 251 are the continuous features from the list `continuous_features`. We can simulate the indices by creating an array with these numbers.

In [ ]:
# get log/ordinal indices
other_indices = indices[:20]
other_indices = np.sort(other_indices)

In [ ]:
# build a list with all feature names we obtained from OHE, append all log and untransformed col names
forest_columns = list(OHE_feature_names) + continuous_features
print("number of encoded/transformed columns by ColumnTransformer:", len(forest_columns))

This means that 251 - 226 = 25 features are untransformed but concatenated in the `forest_estimators`. One of these features are also in the top 20 of most important features, so we also have to add these column names to the list to find the relevant feature names in the indices.

In [ ]:
# find additional feature names
were_transformed_in_forest = OHE_cols + continuous_features
untransformed_in_forest = [col_name for col_name in X_tr_general
                           if col_name not in were_transformed_in_forest]
print("number of additionally concatenated features:", len(untransformed_in_forest))

# append all feature names in the same order as does the ColumnTransformer
all_feature_names = forest_columns + untransformed_in_forest

# print the top 20
for g in np.arange(20):
    print("feature {}\'s name is: {}".format(other_indices[g], all_feature_names[other_indices[g]]))

So the intermediate model will consist of:
* `Fireplace Qu`, this column will be OHE
* `Lot Frontage`, `Lot Area`, `Mas Vnr Area`, `BsmtFin SF 1`, `BSMT Unf SF`, `Total Bsmt SF`, `1st Flr SF`, `2nd Flr SF`, `Gr Liv Area`, `Garage Area`, these columns will be log-transformed
* `Overall Qual`, `Overall Cond`, `Year Built`, `Year Remod/Add`, `Full Bath`, `Kitchen Qual`, `Fireplaces`, `Garage Yr Blt`, `Garage Cars`, will be taken into the model but stay untransformed

In [ ]:
# create the variables, 20 features in the simple model
m2_all_features = ["Fireplace Qu", "Lot Frontage", "Lot Area", "BsmtFin SF 1",
                  "Bsmt Unf SF", "Total Bsmt SF", "1st Flr SF", "2nd Flr SF", "Gr Liv Area", "Garage Area",
                   "Overall Qual", "Overall Cond", "Year Built", "Year Remod/Add", "Full Bath",
                   "Kitchen Qual", "Fireplaces", "Garage Yr Blt", "Garage Cars"]

m2_OHE = m2_all_features[:1] # cols to be OHE
m2_log = m2_all_features[1:11] # cols to be log-transformed
m2_other = m2_all_features[11:] # cols in model but no transformation/encoding
m2_not_cont = ["Fireplace Qu", "Fireplaces", "Kitchen Qual", "Overall Qual", "Overall Cond", "Full Bath",
              "Garage Cars"] # to plot violinplots, excludes ""...Yr Blt"

# visualize cols for this model again
fig, axes = plt.subplots(nrows = 2, ncols = 4, figsize = (20, 8), sharey = True)

for col, ax, position in zip(m2_not_cont, axes.ravel(), np.arange(7)):
    ax = sns.violinplot(train_df[col], train_df["SalePrice"], ax =
                        axes[position // 4][position % 4],
                       # lower bandwidth of kde estimates due to standard deviation of each bin
                       bw = 0.3)
    ax.set_title(col)

fig.suptitle("Distribution of Discrete Features in the Intermediate Model", y = 1.03, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
print("number of negative values in SalePrice:", (train_df["SalePrice"] <= 0).sum())
train_df.loc[train_df["Overall Cond"] == 2, "SalePrice"] # "Overall Cond" stretched up to -100,000 before

With the default `bw = "scott"` calculation of the seaborn violinplots, the violinplots extended up to -100,000.
There are actually no negative values in the whole `train_df` as I have tested during the cleaning steps. However, the kernel density estimate of the violinplot estimates that negative values could be possible due to the standard deviation of the values. [Source from stackoverflow](https://stackoverflow.com/questions/28115637/how-to-better-fit-seaborn-violinplots/28119908#28119908) To prevent such estimates - as negative house prices are not possible -  I try one of the provided solutions of the stackoverflow post and lower the bandwidth of the estimates to `0.3`.

In [ ]:
# plot boxplots of continuous features
fig, axes = plt.subplots(nrows = 3, ncols = 4, figsize = (12, 5))

for col, ax, pos in zip(m2_log, axes.ravel(), np.arange(10)):
    ax = sns.boxplot(np.log1p(train_df[col]), ax = axes[pos // 4][pos % 4]) # log-transformed
    ax.set_title(col)
    
fig.suptitle("Boxplots of Continuous Variables for the Intermediate Model", y = 1.03, size = 15)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# boxplots of year variables
m2_years = ["Year Built", "Year Remod/Add", "Garage Yr Blt"] # group variables

fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (12, 2))

for col, ax, pos in zip(m2_years, axes.ravel(), np.arange(3)):
    ax = sns.boxplot(train_df[col], ax = axes[pos])
    ax.set_title(col)
    
fig.suptitle("Boxplots of Per Year Features", y = 1.05, size = 13)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# distribution of per year variables
fig, axes = plt.subplots(nrows = 1, ncols = 3, figsize = (15, 4))

for col, ax, pos in zip(m2_years, axes.ravel(), np.arange(3)):
    ax = sns.distplot(train_df[col], bins = 20, kde = False, fit = norm, ax = axes[pos])
    ax.set_title(col)
    
fig.suptitle("Distribution of Per Year Variables", y = 1.05, size = 13)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# histograms of continuous variables
fig, axes = plt.subplots(nrows = 3, ncols = 4, figsize = (15, 9))

for col, ax, pos in zip(m2_log, axes.ravel(), np.arange(10)):
    ax = sns.distplot(np.log1p(train_df[col]), bins = 20, kde = False, fit = norm, ax =
                      axes[pos // 4][pos % 4])
    ax.set_title(col)

fig.suptitle("Distribution of Continuous Features in Intermediate Model", y = 1.05, size = 20)
fig.tight_layout() # avoids overlapping labels
plt.show()

In [ ]:
# preprocess TRAIN SET
# slice the dfs
m2_X_tr = train_df[m2_all_features]
m2_X_te = test_df[m2_all_features]
# m1_y_tr, m1_y_te stay the same from the previous model


# define preprocess functions
# OHE, drop the first to avoid numerically instable results, result should be numpy array, not sparse df
m2_encoder = OneHotEncoder(drop = "first", sparse = False) 

# log-transformation of continuous features
m2_transform = FunctionTransformer(np.log1p)

# build preprocessor
m2_preprocessor = ColumnTransformer([
    ("OHE", m2_encoder, m2_OHE), # OHE of columns
    ("log", m2_transform, m2_log) # log transformation
], remainder = "passthrough") # passthrough other features without transformation/encoding, e.g. ordinal

# preprocess m2_X_tr
m2_X_tr_preprocessed = m2_preprocessor.fit_transform(m2_X_tr)

# transform m2_X_te
m2_X_te_preprocessed = m2_preprocessor.transform(m2_X_te)

print("shape of preprocessed m2_X_tr:", m2_X_tr_preprocessed.shape)
print("shape of preprocessed m2_X_te:", m2_X_te_preprocessed.shape)

In [ ]:
# check matrix rank to find collinear features
# add columns of ones to find linear combinations
m2_X1_tr_preprocessed = np.c_[np.ones(shape = m2_X_tr_preprocessed.shape[0]), m2_X_tr_preprocessed]
m2_rank = np.linalg.matrix_rank(m2_X1_tr_preprocessed)
print("matrix rank of m2 should be 24 (incl. col of ones), test:", m2_rank) # no collinear features

# check condition to find nearly collinear features
m2_cond = np.linalg.cond(m2_X1_tr_preprocessed)
print("condition number should be relatively low, test:", m2_cond) # extremely high condition number

#### Choice of Regression model
Because we now have: 
* 20 features in this model
* and not all of them are normally or nearly-normally distributed
* and, especially, an _extremely_ high condition number indicating nearly collinear features
it is advised to use regularization in the regression model. Otherwise, we receive numerically unstable results when the feature matrix gets inverted in the OLS process. The predictive accuracy won't be decreased, but each prediction run we execute will be drastically different due to the extremely large coefficients.

--> Ridge will be used

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as MSE # for grid search validation plot
from sklearn.impute import SimpleImputer

# store results of grid search
gs_results = []

# create loop to try different alphas
for alpha in np.logspace(-10, 10, num = 100): # hyperparameter affect the model on a logarithmic scale
    # create pipeline
    pipe_m2 = Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha))
    ])
    # fit pipeline
    pipe_m2.fit(m2_X_tr_preprocessed, m1_y_tr) # m1_y_tr is already log scaled
    
    # make predictions, append results
    gs_results.append({
        "alpha" : alpha,
        "train mse" : MSE(m1_y_tr, pipe_m2.predict(m2_X_tr_preprocessed)),
        "test mse" : MSE(m1_y_te, pipe_m2.predict(m2_X_te_preprocessed)),
        "train mae" : MAE(np.exp(m1_y_tr), np.exp(pipe_m2.predict(m2_X_tr_preprocessed))), # reverse log to have $
        "test mae" : MAE(np.exp(m1_y_te), np.exp(pipe_m2.predict(m2_X_te_preprocessed))), # reverse log to have $
        "test RSS" : RSS(m1_y_te, pipe_m2.predict(m2_X_te_preprocessed)) # calculate RSS for each alpha
    })

# convert gs_results to DataFrame
gs_results = pd.DataFrame(gs_results)
gs_results.head()

In [ ]:
# plot validation curves
plt.semilogx(gs_results["alpha"], gs_results["train mse"], label = "train mse")
plt.semilogx(gs_results["alpha"], gs_results["test mse"], label = "test mse")
plt.scatter(gs_results.loc[gs_results["test mse"].argmin(), "alpha"],
           gs_results["test mse"].min(), marker = "x", zorder = 10, color = "red")
plt.title("Validation Curves with Ridge Regularizer, Indermediate Model")
plt.xlabel(r"$log(alpha)$")
plt.ylabel("MSE Score")
plt.legend()
plt.show()

It seems like we have an edge solutions here. I have checked the logspaces down until -30, and the values stay the same.

In [ ]:
# find indices of lowest test MSE
min_mse = gs_results["test mse"].idxmin()

# print corresponding MAE, std
print("MAE of Intermediate Model: ${:,.0f}, with regularizing alpha of {:.4f}".format(
    gs_results.loc[min_mse, "test mae"], gs_results.loc[min_mse, "alpha"]))

# print RSS
print("RSS of Intermediate Model: {:.4f}".format(gs_results.loc[min_mse, "test RSS"]))

So in this case it really seems like the edge solution minimizes the error metrics, but our model is overfitting. This can be seen by the significantly larger `test mae` score in comparison to the `train mae`. Additionally - and this is why I chose the Ridge regression - we have a large condition number, and in the ridge model we didn't actually regularize any coefficients since `alpha` is zero at the score which minimizes the error. 

Effectively, we have calculated the MAE with the standard `LinearRegression` module since alpha = 0 when we choose the lowest MAE position.

The error scores might still have decreased in comparison to the simple model, but our model generalizes worse from the data. The error metrics are lower, but only because we overfit the model and predictions are now fitted into/driven by extremes and outliers.

I will try to regularize the model with the `Lasso` $L_{1}$ regularizer, which penalizes the absolute value of coefficients - not the squares as does `Ridge`.
$$
\min_{\vec{w}} L(\vec{w}) \qquad \text{s. t.} \hspace{0.2cm} \sum_{i = 1}^p \left| w_i \right| \leq c
$$

In [ ]:
# build a grid search for Lasso
from sklearn.linear_model import Lasso

# variable to store results
gs_results3 = []

for alpha in np.logspace(-10, 10, num = 200): # hyperparameter affect the coefficients on a log scale
    # create pipeline
    pipe2_m2 = Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha, max_iter = 10000))
    ])
    
    # fit object
    pipe2_m2.fit(m2_X_tr_preprocessed, m1_y_tr) # fit training data
    
    # append results, make predictions
    gs_results3.append({
        "alpha" : alpha,
        "train mse" : MSE(m1_y_tr, pipe2_m2.predict(m2_X_tr_preprocessed)),
        "test mse" : MSE(m1_y_te, pipe2_m2.predict(m2_X_te_preprocessed)),
        "train mae" : MAE(np.exp(m1_y_tr), np.exp(pipe2_m2.predict(m2_X_tr_preprocessed))),
        "test mae" : MAE(np.exp(m1_y_te), np.exp(pipe2_m2.predict(m2_X_te_preprocessed))),
        "test RSS" : RSS(m1_y_te, pipe2_m2.predict(m2_X_te_preprocessed))
    })
    
# convert list to DataFrame
gs_results3 = pd.DataFrame(gs_results3)
gs_results3.head()

In [ ]:
# plot lasso validation curves
plt.semilogx(gs_results3["alpha"], gs_results3["train mse"], label = "train mse")
plt.semilogx(gs_results3["alpha"], gs_results3["test mse"], label = "test mse")
plt.scatter(gs_results3.loc[gs_results3["test mse"].argmin(), "alpha"],
           gs_results3["test mse"].min(), marker = "x", zorder = 10, color = "red")
plt.title("Validation Curves for Lasso Regularizer, Indermediate Model")
plt.xlabel(r"$log(alpha)$")
plt.ylabel("MSE Score")
plt.legend()
plt.show()

We can observe multiple things here:
* There is an optimal alpha where the model doesn't overfit at the same time
* The optimal alpha seems to be around $10^{-2}$, seen by lowest `test mse` score
* The model **overfits** when alpha is lower than $10^{-2}$, seen by the high `test mse` score in comparison to `train mse`. The regularization is too weak in these cases, i.e. coefficients are unrestricted.
* The model **underfits** when alpha gets higher than $10^{-2}$, seen by the strong rise in both `train mse` and `test mse`. In these cases, the regularization gets too strong and the coefficients are too restricted in their ability to generalize, leading to larger error scores. At some point, here at around $10^{-1}$ or $10^0$, all features get zero because alpha is so large that in regularizes everything to zero. Only the intercept term is left, the error scores plateau as a consequence.

In [ ]:
# find indices of lowest test MSE
min_mse3 = gs_results3["test mse"].idxmin()

# print corresponding MAE, std
print("MAE of Intermediate Model (Lasso Regularization): ${:,.0f}, with regularizing alpha of {:.4f}".
      format(gs_results3.loc[min_mse3, "test mae"], gs_results3.loc[min_mse3, "alpha"]))

# print RSS
m2_rss = gs_results3.loc[min_mse3, "test RSS"] # get score in separate variable for R2 calculation
print("RSS of Intermediate Model: {:.2f}".format(m2_rss))

The MAE as well as RSS decreased in this model, meaning that the predictions are now on average only \\$15,827 off when tested with unseen test data. The simple model with only 2 features was off on average \\$23,479, meaning that our predictions got significantly better. 

To continue the comparison from the simple model: The median sale price is \\$169,000, our model makes on average errors of \\$15,827, resulting in an error ratio of $\frac{15,827}{169,000} = 9.37\%$

All in all, this model is definitely preferable to the simple model.

In [ ]:
# make predictions for test data

m2_test_df = test_predictions_df[m2_all_features].copy() # slice df with all needed columns

# use m2_preprocessor to transform this test data
m2_test_features = m2_preprocessor.transform(m2_test_df)

# create pipeline with all preprocessing/scaling steps
m2_test_pipe = Pipeline([
    ("scaler", StandardScaler()),
    # use optimal alpha of training set
    ("lasso", Lasso(alpha = gs_results3.loc[min_mse3, "alpha"], max_iter = 10000))
])

# fit pipeline
m2_test_pipe.fit(m2_X_tr_preprocessed, m1_y_tr) # use training data again
m2_test_pred = m2_test_pipe.predict(m2_test_features) # enter unseen test data here

# create df
m2_test_file = pd.DataFrame({
    "PID" : m1_test_file["PID"],
    "SalePrice" : np.round(np.exp(m2_test_pred), 0).astype(int) # reverse log-transformation of m1_y_tr
})

m2_test_file.to_csv("/Users/Ingo/house-prices/predictions-intermediate-model.csv", index = False)

m2_test_file.head()

___
# Complex Model
This model takes all features into account. 

In [ ]:
# group all relevant features
# continuous_features, OHE_cols are already grouped, now find cols that won't be transformed
not_transformed = [col for col in train_df.columns if col not in continuous_features+OHE_cols]

# remove "SalePrice"
not_transformed.remove("SalePrice")

# create train df
m3_X_tr = train_df[OHE_cols+continuous_features+not_transformed]

# create test df
m3_X_te = test_df[OHE_cols+continuous_features+not_transformed]

In [ ]:
# check whether all sets are separate
print(set(OHE_cols) & set(continuous_features))
print(set(not_transformed) & set(continuous_features))
print(set(not_transformed) & set(OHE_cols))

Because model 2 with 20 variables needed regularization, this model will need regularization, too. Therefore, I will not drop the first column of every OHE column, and instead choose to ignore unknown values. 

In [ ]:
# m1_y_tr and m1_y_te are still valid and already log-transformed for this model
from sklearn.preprocessing import FunctionTransformer

# create preprocessing functions
m3_OHE = OneHotEncoder(handle_unknown = "ignore", sparse = False) # ignore unknown values from test set
m3_log = FunctionTransformer(np.log1p)

# create preprocessor
m3_preprocessor = ColumnTransformer([
    ("OHE", m3_OHE, OHE_cols), # encode categorical features
    ("log", m3_log, continuous_features), # log-transform continuous features
], remainder = "passthrough") # append all column that won't be transformed

# fit on training data, transform training and test data
m3_X_tr_preprocessed = m3_preprocessor.fit_transform(m3_X_tr)
m3_X_te_preprocessed = m3_preprocessor.transform(m3_X_te) # transform with fit of training data

# check shapes
print("train set shape:", m3_X_tr_preprocessed.shape)
print("test set shape:", m3_X_te_preprocessed.shape)

In [ ]:
# add col of ones to train set
m3_X1_tr_preprocessed = np.c_[np.ones(shape = m3_X_tr_preprocessed.shape[0]), m3_X_tr_preprocessed]

# check matrix rank, should be 248 with columns of ones included
m3_rank = np.linalg.matrix_rank(m3_X1_tr_preprocessed)
print("Matrix Rank of Complex Model:", m3_rank)

# check condition number
m3_cond = np.linalg.cond(m3_X1_tr_preprocessed)
print("Condition Number of Complex Model:", m3_cond)

We can see that the matrix isn't full rank, and has an extremely high condition number. Let's try to find the linearly independent cols and drop all other.

In [ ]:
# iterate through m3_X1_tr_preprocessed, calculate matrix rank for each additional column
# if rank didn't get bigger, append the column; this indicates a linearly dependent column

dependent_cols = [] # append cols to be dropped here

# iterate through every column -1, the 1 is added in loop because 0 is starting point
for col in np.arange(1, 286):
    # the array that is checked gets bigger every iteration
    previous_matrix_rank = np.linalg.matrix_rank(m3_X1_tr_preprocessed[:, :col])
    # calculate matrix rank of next col to have comparison
    current_matrix_rank = np.linalg.matrix_rank(m3_X1_tr_preprocessed[:, :col+1])
    # check whether the two ranks stayed the same despite one more col in array
    if previous_matrix_rank == current_matrix_rank:
        dependent_cols.append(col)
dependent_cols

In [ ]:
# drop dependent cols
print("shape before drop:", m3_X1_tr_preprocessed.shape)

m3_X1_tr_preprocessed = np.delete(m3_X1_tr_preprocessed, dependent_cols, axis = 1)
print("after:", m3_X1_tr_preprocessed.shape)

# calculate rank again, should be 244 now
print("matrix rank after dropping dependent columns:", np.linalg.matrix_rank(m3_X1_tr_preprocessed))

# calculate condition number again
print("condition number after dropping dependent columns:", np.linalg.cond(m3_X1_tr_preprocessed))

# drop the column of ones, assign the variable back to m3_X_tr_preprocessed
m3_X_tr_preprocessed = np.delete(m3_X1_tr_preprocessed, 0, axis = 1)

Our feature matrix now has full rank, and we can also see that the condition number has shrunk significantly from $1.97*10^{21}$ to $6,992,251.72$. It is still extremely high, which means that we need to penalize our coefficients again because otherwise predictions would vary significantly with very little change in data.

In the first try, I will use the `Ridge` regression. If the results are similar to the indermediate model, I will use `Lasso` again.

In [ ]:
# add a column of ones to test data to have same indices from training set
m3_X1_te_preprocessed = np.c_[np.ones(shape = m3_X_te_preprocessed.shape[0]), m3_X_te_preprocessed]

# delete dependent columns
m3_X1_te_preprocessed = np.delete(m3_X1_te_preprocessed, dependent_cols, axis = 1)

# drop the column of ones
m3_X_te_preprocessed = np.delete(m3_X1_te_preprocessed, 0, axis = 1)

In [ ]:
# grid search with Ridge

m3_gs_results = [] # store results here

for alpha in np.logspace(-10, 10, num = 200): # hyperparameter affects the coefficients on a log scale
    # create pipeline
    pipe_m3 = Pipeline([
        ("scaler", StandardScaler()),
        ("ridge", Ridge(alpha))
    ])
    # fit pipeline
    pipe_m3.fit(m3_X_tr_preprocessed, m1_y_tr) # fit training data
    
    # append results, make predictions, calculate error scores
    m3_gs_results.append({
        "alpha" : alpha,
        "train mse" : MSE(m1_y_tr, pipe_m3.predict(m3_X_tr_preprocessed)),
        "test mse" : MSE(m1_y_te, pipe_m3.predict(m3_X_te_preprocessed)),
        "train mae" : MAE(np.exp(m1_y_tr), np.exp(pipe_m3.predict(m3_X_tr_preprocessed))), # rescale log data
        "test mae" : MAE(np.exp(m1_y_te), np.exp(pipe_m3.predict(m3_X_te_preprocessed))), # rescale log data
        "RSS" : RSS(m1_y_te, pipe_m3.predict(m3_X_te_preprocessed))
    })
    
# convert list to dataframe
m3_gs_results = pd.DataFrame(m3_gs_results)
m3_gs_results.head()

In [ ]:
# get index of lowest error score
m3_min_mse = m3_gs_results["test mse"].idxmin(axis = 1)

# plot validation curves
plt.semilogx(m3_gs_results["alpha"], m3_gs_results["train mse"], label = "train mse")
plt.semilogx(m3_gs_results["alpha"], m3_gs_results["test mse"], label = "test mse")

# mark lowest error score
plt.scatter(m3_gs_results.loc[m3_min_mse, "alpha"], m3_gs_results.loc[m3_min_mse, "test mse"],
           marker = "x", zorder = 10, color = "red")
plt.title("Valdiation Curves for Ridge Regularizer")
plt.xlabel(r"$log(alpha)$")
plt.ylabel("MSE Score")
plt.legend()
plt.show()

What we can observer here:
* The model is finding an optimum at around $10^1$ 
* The model is overfitting at alphas lower than $10^1$, seen by the higher test error. This is due to little to no regularizations on the coefficients
* The model underfits when alpha is above $10^1$, but not as much as in previous models. We can see this from the rising train and test error scores. This happens because the regularization of coefficients gets even larger, in fact, so large that all coefficients effectively get zero and the only value left is the intercept term. The model plateaus at that stage.

Let's see what the values in \\$ are.

In [ ]:
print("The MAE of the complex model is ${:,.2f}, with an alpha of {:,.4f}".format(
    m3_gs_results.loc[m3_min_mse, "test mae"], m3_gs_results.loc[m3_min_mse, "alpha"]))
print("The corresponding train MAE is ${:,.2f}".format(m3_gs_results.loc[m3_min_mse, "train mae"]))
print("The RSS of the model is:", m3_gs_results.loc[m3_min_mse, "RSS"])

With an error of only \\$1,500 more at the optimum, we got a reasonably good result. Furthermore, the errors are lower than in our intermediate model, so the Ridge regression delivers good results.

Let's still try the Lasso regularizer.

In [ ]:
# lasso regularization

m3_gs_results2 = [] # store results

for alpha in np.logspace(-10, 10, num = 200): # hyperparameter affects model on a log scale
    # create pipeline
    pipe2_m3 = Pipeline([
        ("scaler", StandardScaler()),
        ("lasso", Lasso(alpha, max_iter = 100000))
    ])
    # fit pipeline
    pipe2_m3.fit(m3_X_tr_preprocessed, m1_y_tr)
    
    # make predictions, append results, calculate errors
    m3_gs_results2.append({
        "alpha" : alpha,
        "train mse" : MSE(m1_y_tr, pipe2_m3.predict(m3_X_tr_preprocessed)),
        "test mse" : MSE(m1_y_te, pipe2_m3.predict(m3_X_te_preprocessed)),
        "train mae" : MAE(np.exp(m1_y_tr), np.exp(pipe2_m3.predict(m3_X_tr_preprocessed))),
        "test mae" : MAE(np.exp(m1_y_te), np.exp(pipe2_m3.predict(m3_X_te_preprocessed))),
        "RSS" : RSS(m1_y_te, pipe2_m3.predict(m3_X_te_preprocessed))
    })
    
# convert list to DataFrame
m3_gs_results2 = pd.DataFrame(m3_gs_results2)
m3_gs_results2.head()

In [ ]:
# find lowest alpha
m3_min_mse2 = m3_gs_results2["test mse"].idxmin(axis = 1)

# plot validation curves
plt.semilogx(m3_gs_results2["alpha"], m3_gs_results2["train mse"], label = "train mse")
plt.semilogx(m3_gs_results2["alpha"], m3_gs_results2["test mse"], label = "test mse")

# mark lowest error
plt.scatter(m3_gs_results2.loc[m3_min_mse2, "alpha"], m3_gs_results2.loc[m3_min_mse2, "test mse"],
           marker = "x", color = "red", zorder = 10)
plt.xlabel(r"$log(alpha)$")
plt.ylabel("MSE Score")
plt.title("Validation Curves for Lasso Regularizer, Complex Model")
plt.legend()
plt.show()

We can see here that:
* the optimum is around $10^{-3}$
* the model overfits extremely when the regularization is below an alpha of around $10^{-3}$, this can be seen by the significantly higher test error score
* the model starts underfitting as soon as the alpha regularizes above $10^{-2}$. Both train and test scores rise sharply until effectively all coefficients are penalized to zero, leaving only the intercept term as error score. The graph plateaus at that stage.

Let's check the error scores in \\$

In [ ]:
print("The complex model has an MAE of ${:,.2f} at alpha = {:.4f}".format(
    m3_gs_results2.loc[m3_min_mse2, "test mae"], m3_gs_results2.loc[m3_min_mse2, "alpha"]))
print("The corresponding train MAE is ${:,.2f}".format(m3_gs_results2.loc[m3_min_mse2, "train mae"]))
m3_rss = m3_gs_results2.loc[m3_min_mse2, "RSS"]
print("The RSS of this model is {:.2f}".format(m3_rss))

The MAE as well as RSS decreased again in this model, meaning that the predictions are now on average only \\$12,537,91 off when tested with unseen test data. The simple model with only 2 features was off on average \\$23,479, the indermediate model was off on average \\$15,827 meaning that our predictions got again more precise. However, the jump in additional prediction value decreased, even though we now used 243 features, that is, 220 features more than in the intermediate model. This makes sense, though, because it gets harder and harder to find new, explanatory features that don't dilute/diverge the results from other features.

Sometimes it can happen that one feature explains the trend of the target variable well, and when you take another feature into the model, the prediction results get worse, even though 2 features were used. In such cases, fewer features are preferable. However, this wasn't the case here, at least when judged as a whole. Each model got better at predicting the targets.

To continue the comparison from the simple model and intermediate model: The median sale price is \\$169,000, our model makes on average errors of \\$12,537.91, resulting in an error ratio of $\frac{12,537,91}{169,000} = 7.42\%$

In [ ]:
# calculate baseline y_tr
baseline_mae = MAE(np.exp(m1_y_te), np.full_like(m1_y_te, np.exp(np.median(m1_y_tr))))
print("Baseline MAE is ${:,.2f}".format(baseline_mae))

# calculate adj. R2 for every model
# first, calculate baseline rss with mean of m1_y_tr
baseline_rss = RSS(m1_y_te, np.full_like(m1_y_te, np.mean(m1_y_tr)))
print("Baseline RSS is {:.2f}".format(baseline_rss))

# define function for regular r2
def R2(rss_model, rss_baseline):
    r2 = 1 - (rss_model / rss_baseline)
    return r2

# calculate regular R2 for every model, this is needed as input for adj. R2
m1_R2 = R2(m1_rss, baseline_rss)
m2_R2 = R2(m2_rss, baseline_rss)
m3_R2 = R2(m3_rss, baseline_rss)

print("R2 of Model 1:", m1_R2)
print("R2 of Model 2:", m2_R2)
print("R2 of Model 3:", m3_R2)

We can observe here that the R2 scores consistently get closer to one, meaning that our models explain more and more variation in the data. However, "simple" $R^2$ always get's larger when additional features are taken into the model, even though the column may be completely useless in predicting power. 

In [ ]:
# define function for adj. R2
def adj_R2(R2, n, p):
    adj_r2 = 1 - (1 - R2) * ((n - 1) / (n - p - 1))
    return adj_r2

# calculate adj R2
m1_adj_R2 = adj_R2(m1_R2, 1432, 2) # 1432 entries, 2 features
m2_adj_R2 = adj_R2(m2_R2, 1432, 23) # 1432 entries, 23 features
m3_adj_R2 = adj_R2(m3_R2, 1432, 243) # 1432 entries, 243 features

print("Adj. R2 of Model 1: {:.4f}".format(m1_adj_R2))
print("Adj. R2 of Model 2: {:.4f}".format(m2_adj_R2))
print("Adj. R2 of Model 3: {:.4f}".format(m3_adj_R2))

We can see that the complex model still performs best, but the difference between model 2 and 3 in terms of explaining variances is very marginal. The MAE metric is more suited in these cases to evaluate the model. The complex model's MAE clearly outperforms every other model. 

In [ ]:
# visualize MAE results
mae_list = [54638.71, 23479, 15827, 12537.91]
label_list = ["Baseline", "Simple\nModel", "Intermediate\nModel", "Complex\nModel"]

plt.bar(np.linspace(1, 4, 4), mae_list)
plt.title("Comparison of MAE Scores Among All Models")
plt.xticks(ticks = np.linspace(1, 4, 4), labels = label_list)
plt.xlabel("Model Type")
plt.ylabel("MAE Scores (in Dollars)")
plt.show()

In [ ]:
# create overview of results
results_df = pd.DataFrame({
    "# of Features" : [2, 19, 79],
    "# of Features (OHE)" : [2, 23, 243],
    "MAE ($)" : [23479, 115827, 12537.91],
    "RSS" : [m1_rss, m2_rss, m3_rss],
    "R2" : [m1_R2, m2_R2, m3_R2],
    "adj. R2" : [m1_adj_R2, m2_adj_R2, m3_adj_R2]
}, index = ["Simple Model", "Intermediate Model", "Complex Model"])
results_df

In [ ]:
# make predictions for test data

# group test predictions df
m3_test_df = test_predictions_df[OHE_cols+continuous_features+not_transformed]

# preprocess data
m3_test_features = m3_preprocessor.transform(m3_test_df) # preprocess all data based on training fit

# add a column of ones to test data to have same indices from training set
m3_test_features1 = np.c_[np.ones(shape = m3_test_features.shape[0]), m3_test_features]

# delete dependent columns
m3_test_features1 = np.delete(m3_test_features1, dependent_cols, axis = 1)

# drop the column of ones
m3_test_features = np.delete(m3_test_features1, 0, axis = 1)


# create pipeline
m3_test_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha = m3_gs_results2.loc[m3_min_mse2, "alpha"], max_iter = 100000))
])

# fit pipeline with training data
m3_test_pipe.fit(m3_X_tr_preprocessed, m1_y_tr)

# make prediction
m3_test_pred = m3_test_pipe.predict(m3_test_features)

# create DataFrame with PID
m3_predictions_file = pd.DataFrame({
    "PID" : m1_test_df["PID"],
    "SalePrice" : np.round(np.exp(m3_test_pred), 0).astype(int) # reverse log-transformation of m1_y_tr
})

# save file
m3_predictions_file.to_csv("/Users/Ingo/house-prices/predictions-complex-model.csv", index = False)

# example of predictions
m3_predictions_file.head()